<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Image Cutouts</b> <br>
Contact author(s): Christina Williams <br>
Last verified to run: 2023-09-12 <br>
LSST Science Piplines version: Weekly 2023_35 <br>
Container Size: medium <br>
Targeted learning level: beginner <br>

**Description:** Demonstration of how to use the cutout tool for calexp and deepCoadds.

## 1. Introduction

This is a brief demonstration of how to make a single-filter image cutout using the cutout tool.

The calls to the cutout tool are embedded in the `makeImageCutout` function.

### 1.1 Package Imports

Import general python and astronomy packages, LSST Science Pipelines packages, and image cutout `PyVO` packages.

In [ ]:
import os
import numpy as np
import pandas
import matplotlib.pyplot as plt
from astropy import units as u

from lsst.daf.butler import Butler
import lsst.geom as geom
from lsst.afw.image import Image
from lsst.afw.image import Exposure, ExposureF
import lsst.afw.display as afwDisplay
from lsst.rsp import get_tap_service

from pyvo.dal.adhoc import DatalinkResults, SodaQuery

### 1.2 Define Functions and Parameters

The default option is that image cutouts are saved as `~/DATA/soda-cutout.fits`.

Image cutouts can be renamed and stored in the directory `./cutouts`. If that directory does not exist, create it.

In [ ]:
if os.path.exists('cutouts/'):
    print('Exists: cutouts/')
else:
    os.system('mkdir cutouts')
    print('Created: cutouts/')

Set the `afwDisplay` backend to be `matplotlib`, and the maximum number of rows to display for a `pandas` table to 20.

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')
pandas.set_option('display.max_rows', 20)

Function `plotImage` is simply for convenience of plotting the output.

In [ ]:
def plotImage(exposure: ExposureF, img_opt: dict=None):
    """Plot an image using matplotlib
   
    Parameters
    ----------
    image : `Exposure`
        the image to plot
        
    opts : ``
   
    Returns
    -------
    title : `str` (only if result is not `None`)
        Plot title from string
    """
    
    fig, ax = plt.subplots()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(exposure.image)
    plt.show()


Function `makeImageCutout` provides the cutout service for either `calexp` or `deepCoadds` images.

Note that `SodaQuery.from_resource` creates a instance from a number of records and a Datalink Resource.

In [ ]:
def makeImageCutout(tap_service, ra, dec, cutout_size=0.01, default_filter = 'i', 
                    imtype='deepCoadd', dataId=None, filename=None): 

    """Make a cutout using the cutout tool
   
    Parameters
    ----------
    tap_service : 
       the tap service instantiated with get_tap_service("tap")
    
    ra : `float`
       right asencion of the cutout center, in decimal degrees
    
    dec : `float`
       declination of the cutoutcenter, in decimal degrees
    
    cutout_size : `float`
       the size of the side of the cutout in degrees
    
    default_filter : `str`
       the default filter to use in a deepCoadd dataId if it is not supplied
    
    imtype : `str`
       calexp or deepCoadd (default)
    
    dataId : dict
       dataId to pass to the bulter
       a deepCoadd dataId should have format {'band':'band', 'tract':tract, 'patch':patch}
       a calexp dataId should have format {'visit':visitId, 'detector':detector}
    
    filename : `str`
       to save the cutout as a FITS file in "cutouts/filename.fits"
       otherwise it is saved in "~/DATA/soda-cutout.fits"
    
    Returns
    -------
    
    sodaCutout : `str`
       the filename of the saved image cutout
    
    """

    spherePoint = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)
    
    if imtype == 'calexp':
        query = "SELECT access_format, access_url, dataproduct_subtype, lsst_visit, lsst_detector, " + \
            "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.calexp' " + \
            "AND lsst_visit = " + str(dataId["visit"]) + " " + \
            "AND lsst_detector = " + str(dataId["detector"])
        results = tap_service.search(query)
    else:
        tract = dataId["tract"]
        patch = dataId["patch"]
        if 'band' in dataId:
            band = dataId["band"]
        else:
            band = default_filter
        query = "SELECT access_format, access_url, dataproduct_subtype, lsst_patch, lsst_tract, " + \
            "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.deepCoadd_calexp' " + \
            "AND lsst_tract = " + str(tract) + " " + \
            "AND lsst_patch = " + str(patch) + " " + \
            "AND lsst_band = " + "'" + str(band) + "' "
        results = tap_service.search(query) 

    dataLinkUrl = results[0].getdataurl()
    f"Datalink link service url: {dataLinkUrl}"
    auth_session = service._session
    dl_results = DatalinkResults.from_result_url(dataLinkUrl,session=auth_session)
    f"{dl_results.status}"

    sq = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                                 session=auth_session)

    sq.circle = (spherePoint.getRa().asDegrees()* u.deg,
                 spherePoint.getDec().asDegrees()*u.deg, 
                 cutout_size* u.deg)

    if filename:
        sodaCutout = 'cutouts/'+filename       
    else:
        sodaCutout = os.path.join(os.getenv('HOME'), 'DATA/soda-cutout.fits')

    with open(sodaCutout, 'bw') as f:
        f.write(sq.execute_stream().read())
        
    return sodaCutout

## 2. Make a `deepCoadd` image cutout

Instantiate access to the data butler and the TAP service.

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
registry = butler.registry

In [ ]:
service = get_tap_service("tap")

Define the `dataId` for a given sky coordinate (in this case, the location of a known galaxy cluster) and filter (band).

Use the butler `skyMap` to obtain the `tract` and `patch` info, which are required for the `dataId`, for these coordinates.

In [ ]:
ra = 55.7467 
dec = -32.2862
spherePoint = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)
skymap = butler.get('skyMap')
tract = skymap.findTract(spherePoint)
patch = tract.findPatch(spherePoint)
dataId = {'band':'i', 'tract':tract.tract_id, 'patch': patch.getSequentialIndex()}

Create and display the `deepCoadd` image cutout.

In [ ]:
my_cutout = makeImageCutout(service, ra, dec, dataId=dataId, cutout_size=0.1)
plotImage(ExposureF(my_cutout))

Find the above cutout saved as a FITS file in `~/DATA/soda-cutout.fits`.

## 3. Make a series of `calexp` image cutouts

In this example, for a given `DiaObject`, create small image cutouts (stamps) from the first 10 _i_-band `calexp` images ever obtained by LSST.

This example does not limit cutout creation only to the visits in which the `DiaObject` was detected in the difference-image, so there might not be a source at the cutout's center.

Use the TAP service to obtain the `visitId` and `detector` identifiers for images containing this `DiaObject`.

In [ ]:
diaObjectId = 1253478440036730088

ccdquery = "SELECT TOP 10 dia.coord_ra, dia.coord_dec, " + \
           "dia.diaObjectId,  dia.ccdVisitId, dia.band, " + \
           "cv.visitId, cv.physical_filter, cv.detector, cv.obsStartMJD, " + \
           "cv.expMidptMJD " + \
           "FROM dp02_dc2_catalogs.ForcedSourceOnDiaObject as dia " + \
           "JOIN dp02_dc2_catalogs.CcdVisit as cv ON cv.ccdVisitId = dia.ccdVisitId " + \
           "WHERE dia.diaObjectId = "+str(diaObjectId)+" AND cv.band = 'i'"
    
diaccdsearch = service.search(ccdquery)
forcedSrc = diaccdsearch.to_table()
del diaccdsearch

Sort the visits by MJD.

In [ ]:
wh = np.argsort(forcedSrc['obsStartMJD'])
forcedSrc[wh]

Create 10 small cutouts, display them, and save them with filenames `cutouts/cutout_N.fits` where N will be 0 to 9.

If a pink FutureWarning about the summary field decl not being recognized appears, know that it is OK to ignore and will go away in the future.

In [ ]:
for i in range(10):
    
    ra = forcedSrc['coord_ra'][wh][i]
    dec = forcedSrc['coord_dec'][wh][i]
    spherePoint = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)
    dataId_calexp = {'visit':forcedSrc['visitId'][wh][i], 'detector':forcedSrc['detector'][wh][i]}

    my_cutout = makeImageCutout(service, ra, dec, cutout_size=0.005, imtype='calexp', dataId=dataId_calexp,
                                filename='cutout_'+str(i)+'.fits')
    
    plotImage(ExposureF(my_cutout))